<a href="https://colab.research.google.com/github/cronoxi-cmd/introtopython/blob/main/CST_157_Chapter_Nine_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import pandas as pd
import os
import altair as alt
from IPython.display import display, HTML

# Define some lists of adjectives and products
ADJECTIVES = ["formal", "elegant", "trendy", "vintage", "silk"]
PRODUCTS = ["dress", "handbag", "shoes", "sunglasses", "tie"]


class Product:
    def __init__(self):
        self.name = " ".join([random.choice(ADJECTIVES) for _ in range(3)]) + " " + random.choice(PRODUCTS)
        self.price = round(random.uniform(50, 500), 2)

    def __str__(self):
        return f"{self.name} (${self.price})"


class Customer:
    def __init__(self, name: str):
        self.name = name
        self.basket = Basket()

    def __str__(self):
        return self.name


class Basket:
    def __init__(self):
        self.products: List[Product] = []

    def add(self, product: Product):
        self.products.append(product)

    def calculate_total(self) -> float:
        return round(sum(product.price for product in self.products), 2)

    def __str__(self):
        return "\n".join(str(product) for product in self.products)


class Inventory:
    def __init__(self, num_products: int = 10):
        self.products = [Product() for _ in range(num_products)]

    def display_inventory(self):
        print("Product Inventory:")
        for idx, product in enumerate(self.products, start=1):
            print(f"{idx}. {product}")

    def calculate_total(self) -> float:
        return round(sum(product.price for product in self.products), 2)


class Stand:
    def __init__(self, name: str):
        self.name = name
        self.inventory = Inventory()
        self.customers: List[Customer] = []

    def add_customer(self, customer: Customer):
        self.customers.append(customer)

    def simulate_week(self):
        weekly_sales = self.run()
        weekly_sales['store'] = self.name
        return weekly_sales

    def run(self):
        weekly_sales = []
        for _ in range(7):  # Simulate one week
            daily_sales = self.simulate_day()
            weekly_sales.append(daily_sales)
        return pd.DataFrame(weekly_sales)

    def simulate_day(self):
        daily_sales = {'day': random.choice(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']),
                       'total_sales': self.simulate_customer_activity()}
        return daily_sales

    def simulate_customer_activity(self):
        total_sales = 0
        for customer in self.customers:
            basket = Basket()
            for _ in range(random.randint(1, 5)):
                product = random.choice(self.inventory.products)
                basket.add(product)
            total_sales += basket.calculate_total()
        return total_sales


class Corporation:
    def __init__(self, name: str):
        self.name = name
        self.stands = []

    def add_stand(self, stand: Stand):
        self.stands.append(stand)

    def simulate_year(self):
        all_weekly_sales = []
        for _ in range(52):  # Simulate one year
            weekly_sales = []
            for stand in self.stands:
                weekly_sales.append(stand.simulate_week())
            all_weekly_sales.extend(weekly_sales)
        return pd.concat(all_weekly_sales, ignore_index=True)


# Create a corporation object
corp = Corporation("Vincent's Amazing Corporation")

# Create some store objects and add them to the corporation
store1 = Stand("Crystal Clear Clothing")
store2 = Stand("ZPP Frontier")
store3 = Stand("Hello Outfitters")
corp.add_stand(store1)
corp.add_stand(store2)
corp.add_stand(store3)

# Define a list of customer names
customer_names = ["alex", "jeff", "zoe", "bob", "stefan", "daniel"]

# Add customers to the stores
for name in customer_names:
    store1.add_customer(Customer(name))
    store2.add_customer(Customer(name))
    store3.add_customer(Customer(name))

# Simulate the corporation for a year
yearly_sales = corp.simulate_year()

# Calculate weekly sales for each store
weekly_sales = yearly_sales.groupby(['store', 'day']).sum().reset_index()

# Graph the entire year of sales using Altair
chart = alt.Chart(weekly_sales, width=800, height=300).mark_line().encode(
    x='day',
    y='total_sales',
    color='store'
).properties(
    title='Yearly Sales'
)

# Save the chart as an HTML file
chart_html = 'chart.html'
chart.save(chart_html)

# Read the HTML file content
with open(chart_html, 'r') as file:
    chart_content = file.read()

# Display the Altair chart
display(HTML(chart_content))

# Create a DataFrame from the weekly sales data
df = pd.DataFrame(weekly_sales)

# Calculate the variance for each store
grouped_variance = df.groupby('store')['total_sales'].var().rename("variance")

# Rank the stores by variance
ranked_stores = grouped_variance.sort_values(ascending=False)

# Print the ranked stores
print(ranked_stores)

# Display the DataFrame
display(df)

store
Hello Outfitters          4.618435e+09
ZPP Frontier              2.072023e+09
Crystal Clear Clothing    6.167096e+08
Name: variance, dtype: float64


,store,day,total_sales
0,Crystal Clear Clothing,Fri,257268.73
1,Crystal Clear Clothing,Mon,186059.20
2,Crystal Clear Clothing,Sat,253187.02
3,Crystal Clear Clothing,Sun,234847.79
4,Crystal Clear Clothing,Thu,209115.59
5,Crystal Clear Clothing,Tue,229390.86
6,Crystal Clear Clothing,Wed,235582.70
7,Hello Outfitters,Fri,278231.74
8,Hello Outfitters,Mon,322426.06
9,Hello Outfitters,Sat,279587.94
